In [1]:
import sklearn
import pickle
import praw
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('all')
from nltk.corpus import stopwords

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\h

[nltk_data]    |   Package ptb is already up-to-date!
[nltk_data]    | Downloading package product_reviews_1 to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package product_reviews_1 is already up-to-date!
[nltk_data]    | Downloading package product_reviews_2 to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package product_reviews_2 is already up-to-date!
[nltk_data]    | Downloading package pros_cons to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package pros_cons is already up-to-date!
[nltk_data]    | Downloading package qc to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]  

[nltk_data]    |   Package bllip_wsj_no_aux is already up-to-date!
[nltk_data]    | Downloading package word2vec_sample to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package word2vec_sample is already up-to-date!
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package panlex_swadesh is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |  

In [2]:
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))


In [3]:
sym1 = re.compile('[/(){}\[\]\|@,;]')
sym2 = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text
    text = text.lower()
    text = sym1.sub(' ',text)
    text = sym2.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

In [4]:
reddit = praw.Reddit(client_id='bHR-mvDC2TTuww', 
                     client_secret='2UTEBovi2dvNHrDNTY2I6KAInjU', 
                     user_agent='myfirstapp', 
                     username='EltronProtoN',  
                     password='samosa03')


In [5]:
def detect_flair(id):
    submission = reddit.submission(id=id)
    data = {}
    
    data['title'] = submission.title
    data['url'] = submission.url
    
    submission.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in submission.comments:
        comment = comment + ' ' + top_level_comment.body
    data["comment"] = comment
    data['title'] = clean_text(data['title'])
    data['comment'] = clean_text(data['comment'])
    data['combine'] = data['title'] + data['comment'] + data['url']
  
    return loaded_model.predict([data['combine']])

In [6]:
subreddit = reddit.subreddit('india')

for submission in subreddit.top(limit = 10):
    print(detect_flair(submission.id),submission.permalink)

['AskIndia'] /r/india/comments/981o7s/will_donate_thrice_the_number_of_upvotes_amount/
['AskIndia'] /r/india/comments/6f10op/indian_reply_to_nytimes_cartoon_on_paris_climate/
['AskIndia'] /r/india/comments/8pymkp/the_essence_of_the_indian_soap_opera_distilled/
['AskIndia'] /r/india/comments/4s5bpn/tragedy_of_india/
['AskIndia'] /r/india/comments/9dt64s/if_you_are_not_moved_by_this_picture_i_wish_i_had/


RequestException: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Max retries exceeded with url: /api/morechildren/?raw_json=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000022B0E1622B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))